In [2]:
import os
import io
import pickle
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import time

class GoogleDriveUploader:
    def __init__(self, credentials_file='credentials.json', token_file='token.pickle'):
        """
        구글 드라이브 업로더 초기화 (OAuth 2.0 사용자 인증)
        
        Args:
            credentials_file (str): OAuth 2.0 클라이언트 자격 증명 JSON 파일
            token_file (str): 액세스 토큰 저장 파일
        """
        self.credentials_file = credentials_file
        self.token_file = token_file
        self.scopes = ['https://www.googleapis.com/auth/drive.file']
        self.service = self._authenticate()
    
    def _authenticate(self):
        """OAuth 2.0으로 구글 드라이브 API 인증"""
        creds = None
        
        # 기존 토큰 파일이 있다면 로드
        if os.path.exists(self.token_file):
            with open(self.token_file, 'rb') as token:
                creds = pickle.load(token)
        
        # 유효한 자격 증명이 없다면 사용자 인증 진행
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                try:
                    creds.refresh(Request())
                    print("🔄 토큰 갱신 완료")
                except Exception as e:
                    print(f"⚠️  토큰 갱신 실패, 재인증 필요: {e}")
                    creds = None
            
            if not creds:
                if not os.path.exists(self.credentials_file):
                    print(f"❌ 자격 증명 파일을 찾을 수 없습니다: {self.credentials_file}")
                    print("📝 Google Cloud Console에서 OAuth 2.0 클라이언트 ID를 생성하고 JSON 파일을 다운로드하세요.")
                    return None
                
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.credentials_file, self.scopes)
                creds = flow.run_local_server(port=0)
                print("✅ 사용자 인증 완료")
            
            # 토큰을 파일에 저장
            with open(self.token_file, 'wb') as token:
                pickle.dump(creds, token)
                print("💾 토큰 저장 완료")
        
        try:
            service = build('drive', 'v3', credentials=creds)
            print("✅ 구글 드라이브 API 인증 성공")
            return service
        except Exception as e:
            print(f"❌ API 서비스 생성 실패: {e}")
            return None
    
    def extract_folder_id_from_url(self, drive_url):
        """
        구글 드라이브 URL에서 폴더 ID 추출
        
        Args:
            drive_url (str): 구글 드라이브 폴더 URL
            
        Returns:
            str: 폴더 ID 또는 None
        """
        try:
            if '/folders/' in drive_url:
                folder_id = drive_url.split('/folders/')[1].split('?')[0].split('/')[0]
                return folder_id
            else:
                print("❌ 올바르지 않은 구글 드라이브 폴더 URL입니다.")
                return None
        except Exception as e:
            print(f"❌ URL에서 폴더 ID 추출 실패: {e}")
            return None
    
    def get_folder_info(self, folder_id):
        """
        폴더 정보 조회
        
        Args:
            folder_id (str): 폴더 ID
            
        Returns:
            dict: 폴더 정보 또는 None
        """
        try:
            folder_info = self.service.files().get(
                fileId=folder_id,
                fields='id, name, parents'
            ).execute()
            return folder_info
        except Exception as e:
            print(f"❌ 폴더 정보 조회 실패: {e}")
            return None
    
    def upload_text_file(self, content, filename, folder_id=None):
        """
        텍스트 내용을 파일로 구글 드라이브에 업로드
        
        Args:
            content (str): 업로드할 텍스트 내용
            filename (str): 파일명
            folder_id (str): 업로드할 폴더 ID (기본값: None - 루트 폴더)
            
        Returns:
            str: 업로드된 파일 ID 또는 None
        """
        if not self.service:
            print("❌ 인증되지 않은 상태입니다.")
            return None
            
        try:
            # 텍스트 내용을 BytesIO 객체로 변환
            file_content = io.BytesIO(content.encode('utf-8'))
            
            # 파일 메타데이터 설정
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            # 미디어 업로드 객체 생성
            media = MediaIoBaseUpload(
                file_content, 
                mimetype='text/plain',
                resumable=True
            )
            
            # 파일 업로드 실행
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None
    
    def upload_local_file(self, file_path, folder_id=None, new_filename=None):
        """
        로컬 파일을 구글 드라이브에 업로드
        
        Args:
            file_path (str): 업로드할 로컬 파일 경로
            folder_id (str): 업로드할 폴더 ID (기본값: None - 루트 폴더)
            new_filename (str): 새 파일명 (기본값: None - 원본 파일명 사용)
            
        Returns:
            str: 업로드된 파일 ID 또는 None
        """
        if not self.service:
            print("❌ 인증되지 않은 상태입니다.")
            return None
            
        try:
            if not os.path.exists(file_path):
                print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
                return None
            
            # 파일명 설정
            filename = new_filename if new_filename else os.path.basename(file_path)
            
            # 파일 메타데이터 설정
            file_metadata = {'name': filename}
            if folder_id:
                file_metadata['parents'] = [folder_id]
            
            # 파일 읽기 및 업로드
            with open(file_path, 'rb') as file_content:
                media = MediaIoBaseUpload(
                    io.BytesIO(file_content.read()),
                    mimetype='application/octet-stream',
                    resumable=True
                )
            
            # 파일 업로드 실행
            file = self.service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id, name, webViewLink'
            ).execute()
            
            file_id = file.get('id')
            file_name = file.get('name')
            web_link = file.get('webViewLink')
            
            print(f"✅ 파일 업로드 성공!")
            print(f"   📄 파일명: {file_name}")
            print(f"   🆔 파일 ID: {file_id}")
            print(f"   🔗 링크: {web_link}")
            
            return file_id
            
        except Exception as e:
            print(f"❌ 파일 업로드 실패: {e}")
            return None

def main():
    """메인 함수 - 사용 예제"""
    
    # OAuth 2.0 클라이언트 자격 증명 JSON 파일 경로
    CREDENTIALS_FILE = '/Users/admin/DEVELOP/Y2025/GITHUB/my-talk/.key/oauth2_client_secret2.json'  # Google Cloud Console에서 다운로드한 파일
    
    # 대상 폴더 URL
    TARGET_FOLDER_URL = 'https://drive.google.com/drive/folders/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_'
    
    # 업로드할 텍스트 내용
    text_content = f"""
안녕하세요! 이것은 구글 드라이브 업로드 테스트입니다.

이 파일은 파이썬 프로그램을 통해 자동으로 업로드되었습니다.
- OAuth 2.0 사용자 인증 사용
- Google Drive API v3 활용
- 지정된 폴더에 저장

대상 폴더 ID: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_
작성 시간: {time.strftime('%Y년 %m월 %d일 %H:%M:%S')}

이 방식은 개인 구글 드라이브에 정상적으로 업로드됩니다!
"""
    
    try:
        # 구글 드라이브 업로더 인스턴스 생성
        uploader = GoogleDriveUploader(CREDENTIALS_FILE)
        
        if not uploader.service:
            print("❌ 업로더 초기화 실패")
            return
        
        # 1. URL에서 폴더 ID 추출
        folder_id = uploader.extract_folder_id_from_url(TARGET_FOLDER_URL)
        
        if not folder_id:
            print("❌ 폴더 ID 추출 실패")
            return
        
        print(f"📁 대상 폴더 ID: {folder_id}")
        
        # 2. 폴더 정보 확인
        folder_info = uploader.get_folder_info(folder_id)
        if folder_info:
            print(f"📁 대상 폴더명: {folder_info.get('name', '알 수 없음')}")
        
        # 3. 텍스트 내용을 파일로 업로드
        filename = f"업로드_테스트_{int(time.time())}.txt"
        file_id = uploader.upload_text_file(
            content=text_content,
            filename=filename,
            folder_id=folder_id
        )
        
        # 4. 로컬 파일 업로드 예제 (선택사항)
        # local_file_path = "example.txt"
        # if os.path.exists(local_file_path):
        #     uploader.upload_local_file(local_file_path, folder_id)
        
        if file_id:
            print("\n🎉 모든 작업이 성공적으로 완료되었습니다!")
            print(f"📁 업로드된 위치: {TARGET_FOLDER_URL}")
        else:
            print("\n❌ 파일 업로드가 실패했습니다.")
            
    except Exception as e:
        print(f"❌ 프로그램 실행 중 오류 발생: {e}")

if __name__ == "__main__":
    main()

✅ 구글 드라이브 API 인증 성공
📁 대상 폴더 ID: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_
❌ 폴더 정보 조회 실패: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_?fields=id%2C+name%2C+parents&alt=json returned "File not found: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_.". Details: "[{'message': 'File not found: 1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">
✅ 파일 업로드 성공!
   📄 파일명: 업로드_테스트_1755926356.txt
   🆔 파일 ID: 1pJVT0rhzZV9qffx-KqGhZM5NPzVOuZIm
   🔗 링크: https://drive.google.com/file/d/1pJVT0rhzZV9qffx-KqGhZM5NPzVOuZIm/view?usp=drivesdk

🎉 모든 작업이 성공적으로 완료되었습니다!
📁 업로드된 위치: https://drive.google.com/drive/folders/1ADawmHJ72tr6JgVQIsZ4j7pDS6cjKgi_
